In [1]:
import pandas as pd
import numpy as np
from functools import partial
import mysql.connector



#Cargas de CSV
dataframeAtletas = pd.read_csv("Olympic_Athlete.csv")
juegosOlimpicos = pd.read_csv("Olympic_Athlete_Event_Results.csv")



#Funcion para convertir a entero o que devuelva NA (1 para entero  2 para float)
def convertirNumero(valor , numero:int):
    try:
        if numero ==1:
            numero = int(valor)
        elif numero == 2:
            numero = float(valor)      
        return numero    
    except ValueError:
        return pd.NA

#Confuncion para devolver NA

def convertirNA(valor):
    if valor == "na":
        return pd.NA
    else:
        return valor
              


#establecer  elsegundo  parametro 
convertirFloat = partial(convertirNumero, numero=2)

#mapeo utiliza la funcion convertirFloat
columna = list( map ( convertirFloat, dataframeAtletas["height"] ))
#mapeo utiliza la funcion convertirFloat
columnaWeight = list( map ( convertirFloat, dataframeAtletas["weight"] ))
#mapeo para devolver NA 
columnaMedalla = list(map(convertirNA, juegosOlimpicos["medal"]))

#reemplazo de nuevas columnas
juegosOlimpicos["medal"] = columnaMedalla
dataframeAtletas["height"] = columna    
dataframeAtletas["weight"] = columnaWeight     

#suma los datos con na 
dataframeAtletas.iloc[[0]]  # sacar x fila
dataframeAtletas.iloc[1:10]  # sacar rangos de filas
dataframeAtletas["height"].mean()
dataframeAtletas.shape
juegosOlimpicos["medal"].value_counts() #conteo de elemntos por distinción

#eliminar columnas born, description,description
columnasEliminarAtletas = ["description", "special_notes" ]
dataframeAtletas = dataframeAtletas.drop(columnasEliminarAtletas, axis=1)


columnasEliminarJuegos = ["description", "special_notes"]
juegosOlimpicos = juegosOlimpicos.drop("isTeamSport",axis=1)


#left join  entre 2 tablas del evento olimpico y el atleta "
tablaJuegosOlimpicos = pd.merge(juegosOlimpicos, dataframeAtletas, on="athlete_id", how="left")
tablaJuegosOlimpicos.groupby("edition").count().sort_values(by="edition_id", ascending=False)
MXJuegoOlimpicos = tablaJuegosOlimpicos.loc[tablaJuegosOlimpicos["country_noc_x"]== "MEX" ]
#filtro x sexo
MxMujeres = MXJuegoOlimpicos[MXJuegoOlimpicos['sex'] =="Female"]
MxHombres = MXJuegoOlimpicos[MXJuegoOlimpicos['sex'] =="Male"]
#agrupar por año
grupoMujeres = MxMujeres.groupby("edition")["sport"].count()
grupoHombres = MxHombres.groupby("edition")["sport"].count()

#union de tablas outer
participacionesXSexo = pd.merge(grupoMujeres, grupoHombres, on="edition", how="outer").reset_index()

#renombrar columnas
participacionesXSexo.rename(columns={
    "sport_x":"Mujeres",
    "sport_y":"Hombres"   
}, inplace=True)

#extraer palabra y crear columna de tipo entero
participacionesXSexo["año"]= participacionesXSexo["edition"].str.split().str.get(0).astype(int)
#rellenar los Na con 0
participacionesXSexo.fillna(0, inplace=True)
#suma de columnas
participacionesXSexo["participaciones_totales"] =  participacionesXSexo["Hombres"] + participacionesXSexo["Mujeres"]
#pasar a enterno
participacionesXSexo["participaciones_totales"].astype(int)
#ordenar por año
participacionesXSexo.sort_values(by="año",ascending=True)
#Filtro solo los juegos de Verano
participacionesXSexo = participacionesXSexo.loc[participacionesXSexo["edition"].str.contains("Summer")]
participacionesXSexo.sort_values(by="participaciones_totales",ascending=False).reset_index()


,index,edition,Mujeres,Hombres,año,participaciones_totales
0,6,1968 Summer Olympics,125.0,387,1968,512.0
1,7,1972 Summer Olympics,72.0,220,1972,292.0
2,22,2020 Summer Olympics,85.0,109,2020,194.0
3,13,1992 Summer Olympics,43.0,129,1992,172.0
4,1,1948 Summer Olympics,9.0,161,1948,170.0
5,10,1984 Summer Olympics,42.0,116,1984,158.0
6,17,2004 Summer Olympics,72.0,78,2004,150.0
7,11,1988 Summer Olympics,31.0,119,1988,150.0
8,20,2016 Summer Olympics,56.0,86,2016,142.0
9,8,1976 Summer Olympics,15.0,125,1976,140.0


In [4]:


#Se crea columna año
MXJuegoOlimpicos["año"]= MXJuegoOlimpicos["edition"].str.split().str.get(0).astype(int)
#Filtro Solo donde se ganan medallas
MedallasMexico = MXJuegoOlimpicos[~MXJuegoOlimpicos["medal"].isna()]

MedallasGanadas= MedallasMexico[["result_id","medal","sex"]].groupby(["medal","sex"]).nunique().reset_index()
MedallasGanadas



C:\Users\TEST\AppData\Local\Temp\ipykernel_10504\4156013760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MXJuegoOlimpicos["año"]= MXJuegoOlimpicos["edition"].str.split().str.get(0).astype(int)


,medal,sex,result_id
0,Bronze,Female,11
1,Bronze,Male,26
2,Gold,Female,2
3,Gold,Male,11
4,Silver,Female,7
5,Silver,Male,17


In [22]:

resultados = pd.read_csv("Olympic_Results.csv")
#seleccion de columans del archivo csv
resultados = pd.DataFrame(resultados[["result_id", "sport","event_title","result_description"]])
#filtro que tengas medallas y año 
MXJuegoOlimpicos2020 = MXJuegoOlimpicos[(~MXJuegoOlimpicos["medal"].isna()) & (MXJuegoOlimpicos["año"]==2016) ].reset_index()
#"agrupar por evento"
MXJuegoOlimpicos2020 =MXJuegoOlimpicos2020[["result_id","medal"]].groupby(["result_id","medal"]).nunique()
#se genera el merge con la descripcion del evento
medallasMx2020 = pd.merge(MXJuegoOlimpicos2020, resultados, on="result_id",how ="left")
medallasMx2020


,result_id,sport,event_title,result_description
0,351482,Taekwondo,"Heavyweight (>67 kilograms), Women",The women’s heavyweight division contained the...
1,351850,Modern Pentathlon,"Individual, Men",Thirty-six athletes took part in the men’s eve...
2,353784,Diving,"Platform, Men",There was some concern amongst British diving ...
3,356522,Boxing,"Middleweight (≤75 kilograms), Men",Just before the opening ceremony of the Rio Ol...
4,359110,Athletics,"20 kilometres Race Walk, Women",Chinese walker Liu Hong had finished fourth at...


In [18]:
#agrupación
medallasTotales = MedallasMexico[["año","result_id","medal"]].groupby(["año","result_id","medal"]).sum().reset_index()
#doy valor a la medalla para que me lo sume"
medallasTotales["cantidad"] = 1 
#filtar y ordenar
filtro= medallasTotales[medallasTotales["año"]==2016].sort_values(by = "medal")

filtro.groupby("medal").sum().reset_index().drop(columns=["año"])



,medal,result_id,cantidad
0,Bronze,708372,2
1,Silver,1064376,3


In [19]:
#mosrar el historico de medallas

medalleroMX = MedallasMexico.groupby(["año","result_id"]).count().reset_index()
#doy parar a las medallas
medalleroMX["cantidad"]= 1
medalleroMX =medalleroMX[["año", "cantidad"]]
medalleroMX.groupby("año").sum()



,cantidad
año,
1900,1
1932,2
1936,3
1948,4
1952,1
1956,2
1960,1
1964,1
1968,9


In [21]:
#mostrar x año las copetencias que llegarona cuarto lugar


#filtrar  por el posicion 4
posicion4= MXJuegoOlimpicos[MXJuegoOlimpicos["pos"]=="4"].reset_index()
#seleccion de columnas
deportesPosicion4= posicion4[["año","result_id"]]
dfDesportesAgrupado = deportesPosicion4.groupby(["año", "result_id"]).nunique().reset_index()

dfDesportesAgrupado.groupby("año").count()


,result_id
año,
1932,1
1948,2
1952,1
1960,2
1964,1
1968,9
1972,1
1980,1
1992,1


In [7]:

#posicion 5
posicion5= MXJuegoOlimpicos[MXJuegoOlimpicos["pos"]=="5"].reset_index()
deportesPosicion5= posicion5[["sport","result_id"]]
deportesPosicion5.groupby("sport").nunique().sort_values(by="result_id",ascending=False).reset_index()

,sport,result_id
0,Diving,10
1,Athletics,7
2,Archery,2
3,Modern Pentathlon,2
4,Tennis,2
5,Weightlifting,2
6,Artistic Gymnastics,1
7,Basketball,1
8,Cycling Track,1
9,Shooting,1


In [8]:
#posicion 4"
deportesPosicion4= posicion4[["sport","result_id"]]


datosInsertar = deportesPosicion4.groupby("sport").nunique().sort_values(by="result_id",ascending=False).reset_index()


datosInsertar

,sport,result_id
0,Diving,13
1,Tennis,4
2,Archery,3
3,Athletics,3
4,Artistic Gymnastics,2
5,Shooting,2
6,Basketball,1
7,Basque pelota,1
8,Canoe Sprint,1
9,Football,1
